In [ ]:
import numpy as np
import pandas as pd
import gensim
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re

# nltk.download('stopwords')
# nltk.download('wordnet')
# !pip install -U gensim
# nltk.download('punkt')
stopwords = stopwords.words('english')

In [ ]:
#model training for new york times 

df = pd.read_csv('nyt.csv', sep=',')

columns = ['pub_date', 'web_url', 'content']
df = df[columns]

df.drop_duplicates(subset ="content", inplace = True)
df.dropna(inplace=True)

print('After droping duplicate rows and unneccesary columns\n Number of Columns: {}\
        , rows: {}'.format(df.shape[1], df.shape[0]) )

In [ ]:
# initalizing the werdnet lemmatizer
lm = WordNetLemmatizer()
# creating a new column to store rows after processing
df['post_processed'] = np.nan

stopwords.extend(['advertisement', 'supported'])
def processing(content):

    content = content.split(' ')
#     removing stopwords
    content = [word.strip().lower() for word in content if word.lower() not in stopwords]
#     removing these punctuations from tokens like it will convert the word mode? into mode
    rx = re.compile('([&#.:?!-()])*')
    content = [rx.sub('', word) for word in content]
    
    content = [word for word in content if len(word)>1 and word.isalpha()]
#   stemming the words  
    content = [lm.lemmatize(word) for word in content]

    return ' '.join(content)

for i in range(len(df)):
    
    if 'opinion' not in df.iloc[i,1]:
        df.iloc[i,-1] = processing(df.iloc[i,2])

df.dropna(inplace=True)
print('After Removing Opinion articles, rows are reduced to : ', df.shape[0])
df.head()

In [ ]:
import gensim.corpora as corpora
#decomposing sentences into tokens 
tokens = [sentence.split(' ') for sentence in df['post_processed'] ]

# Create Dictionary
dct = corpora.Dictionary(tokens)
print('Unique words before filtering/after pre-processing', len(dct))
# no_below= 30
dct.filter_extremes(no_below= 30, no_above=0.25 )
print('Unique words after filtering', len(dct))
# Create Corpus
corpus = [dct.doc2bow(token) for token in tokens]

lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dct, num_topics=7, 
                                           random_state=100, chunksize=128, passes=10 )

In [ ]:
from gensim.test.utils import datapath

# temp_file = datapath("lda.model")
lda_model.save('lda.model')
# Load a potentially pretrained model from disk.
# lda_model1 = gensim.models.LdaModel.load('lda.model')
dct.save('dictionary')